In [ ]:
import requests
import json
import pymysql
import time
import traceback
def getdata():#从腾讯接口爬取数据，并存到字典及列表中
    url = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_h5"
    header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3947.100 Safari/537.36"
    }
    r1 = requests.get(url,headers = header)
    res1 = json.loads(r1.text)
    data_all = json.loads(res1["data"])
    
    url2 = "https://view.inews.qq.com/g2/getOnsInfo?name=disease_other"
    r2 = requests.get(url2,headers = header)
    res2 = json.loads(r2.text)
    odata_all = json.loads(res2["data"])
    
    country_history = {}#历史数据，全国
    for i in odata_all["chinaDayList"]:
        ds ="2020."+ i["date"]
        tup =time.strptime(ds,"%Y.%m.%d")
        ds =time.strftime("%Y-%m-%d",tup)
        confirm =i["confirm"]
        heal=i["heal"]
        dead=i["dead"]
        country_history[ds] ={"confirm":confirm, "heal":heal, "dead":dead}
    for i in odata_all["chinaDayAddList"]:
        ds ="2020."+ i["date"]
        tup =time.strptime(ds,"%Y.%m.%d")
        ds =time.strftime("%Y-%m-%d",tup)
        confirm =i["confirm"]
        country_history[ds].update({"confirm_add":confirm})
        
    details =[]#各省数据
    update_time=data_all["lastUpdateTime"]
    data_province=data_all["areaTree"][0]["children"]
    for pro_infos in data_province:
        province=pro_infos["name"]
        confirm=pro_infos["total"]["confirm"]
        confirm_add=pro_infos["today"]["confirm"]
        heal=pro_infos["total"]["heal"]
        dead=pro_infos["total"]["dead"]
        details.append([update_time,province,confirm,confirm_add,heal,dead])
    
    country_now =[]#全国如今数据
    update_time =data_all["lastUpdateTime"]
    nowConfirm =data_all["chinaTotal"]["nowConfirm"]
    suspect =data_all["chinaTotal"]["suspect"]
    nowSevere =data_all["chinaTotal"]["nowSevere"]
    confirm =data_all["chinaTotal"]["confirm"]
    heal =data_all["chinaTotal"]["heal"]
    dead =data_all["chinaTotal"]["dead"]    
    nowConfirm_add =data_all["chinaAdd"]["nowConfirm"]
    suspect_add =data_all["chinaAdd"]["suspect"]
    nowSevere_add =data_all["chinaAdd"]["nowSevere"]
    confirm_add =data_all["chinaAdd"]["confirm"]
    heal_add =data_all["chinaAdd"]["heal"]
    dead_add =data_all["chinaAdd"]["dead"]
    country_now.append([update_time,nowConfirm,suspect,nowSevere,confirm,heal,dead,nowConfirm_add,suspect_add,nowSevere_add,confirm_add,heal_add,dead_add])
    
    province_history = []#历史数据，各省
    ds=time.strftime("%Y-%m-%d")
    data_province=data_all["areaTree"][0]["children"]
    for pro_infos in data_province:
        province=pro_infos["name"]
        confirm=pro_infos["total"]["confirm"]
        confirm_add=pro_infos["today"]["confirm"]
        heal=pro_infos["total"]["heal"]
        dead=pro_infos["total"]["dead"]
        province_history.append([ds,province,confirm,confirm_add,heal,dead])
    
    return country_history,details,country_now,province_history

In [ ]:
def get_conn():#把常用的函数封装，打开关闭数据库链接
    conn=pymysql.connect(host="localhost",
                     user="root",
                     password="123456",
                     db="covtest",
                     charset="utf8")
    cursor=conn.cursor()
    return conn,cursor

def close_conn(conn,cursor):
    if cursor:
        cursor.close()
    if conn:
        conn.close()

In [ ]:
def create_table():#在数据库中创建四张表，其实建表我是直接用命令行的,不过测试了一下函数可用
    cursor=None
    conn=None 
    try:
        conn,cursor=get_conn()
        sql1 ="""
        CREATE TABLE `country_history` (
            `ds` datetime NOT NULL COMMENT '日期',
            `confirm` int(11) DEFAULT NULL COMMENT '累计确诊',
            `heal` int(11) DEFAULT NULL COMMENT '累计治愈',
            `dead` int(11) DEFAULT NULL COMMENT '累计死亡',
            `confirm_add` int(11) DEFAULT NULL COMMENT '新增确诊',
            PRIMARY KEY (`ds`) USING BTREE
            )ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
               """
        cursor.execute(sql1)
        #conn.commit()sql1 ="""
        print("建表country_history成功")
        
        sql2 ="""
        CREATE TABLE `details`(
                  `id` int(11) NOT NULL AUTO_INCREMENT,
                  `update_time` datetime DEFAULT NULL COMMENT '数据更新时间',
                  `province` varchar(50) DEFAULT NULL COMMENT '省名',
                  `confirm` int(11) DEFAULT NULL COMMENT '累计确诊',
                  `confirm_add` int(11) DEFAULT NULL COMMENT '新增确诊',
                  `heal` int(11) DEFAULT NULL COMMENT '累计治愈',
                  `dead` int(11) DEFAULT NULL COMMENT '累计死亡',
                  PRIMARY KEY (`id`) 
                  )ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
               """
        cursor.execute(sql2)
        #conn.commit()
        print("建表details成功")
        
        sql3 ="""
        CREATE TABLE `country_now` (
                  `id` int(11) NOT NULL AUTO_INCREMENT,
                  `update_time` datetime DEFAULT NULL COMMENT '数据更新时间',
                  `nowConfirm` int(11) DEFAULT NULL COMMENT '现存确诊',
                  `suspect` int(11) DEFAULT NULL COMMENT '现存疑似',
                  `nowSevere` int(11) DEFAULT NULL COMMENT '现存重症',
                  `confirm` int(11) DEFAULT NULL COMMENT '累计确诊',
                  `heal` int(11) DEFAULT NULL COMMENT '累计治愈',
                  `dead` int(11) DEFAULT NULL COMMENT '累计死亡',
                  `nowConfirm_add` int(11) DEFAULT NULL COMMENT '新增现存确诊',
                  `suspect_add` int(11) DEFAULT NULL COMMENT '新增现存疑似',
                  `nowSevere_add` int(11) DEFAULT NULL COMMENT '新增现存重症',
                  `confirm_add` int(11) DEFAULT NULL COMMENT '新增累计确诊',
                  `heal_add` int(11) DEFAULT NULL COMMENT '新增累计治愈',
                  `dead_add` int(11) DEFAULT NULL COMMENT '新增累计死亡',
                  PRIMARY KEY (`id`)
                  )ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
               """
        cursor.execute(sql3) 
        #conn.commit()
        print("建表country_now成功")
        
        sql4 ="""
        CREATE TABLE `province_history` (
                  `id` int(11) NOT NULL AUTO_INCREMENT,
                  `ds` datetime NOT NULL COMMENT '日期',
                  `province` varchar(50) DEFAULT NULL COMMENT '省名',
                  `confirm` int(11) DEFAULT NULL COMMENT '累计确诊',
                  `confirm_add` int(11) DEFAULT NULL COMMENT '新增累计确诊',
                  `heal` int(11) DEFAULT NULL COMMENT '累计治愈',
                  `dead` int(11) DEFAULT NULL COMMENT '累计死亡',
                   PRIMARY KEY (`id`)
                  )ENGINE=InnoDB DEFAULT CHARSET=utf8mb4;
               """
        cursor.execute(sql4)
        #conn.commit()
        print("建表province_history成功")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn,cursor)

In [ ]:
def update_details():#更新details表，因为用了insert命令，所以取出时必须判断最新时间为最新数据
    cursor=None
    conn=None
    try:
        li=getdata()[1]
        conn,cursor=get_conn()
        sql ="insert into details(update_time,province,confirm,confirm_add,heal,dead) values(%s,%s,%s,%s,%s,%s)"
        sql_query='select %s=(select update_time from details order by id desc limit 1)'
        cursor.execute(sql_query,li[0][0])
        if not cursor.fetchone()[0]:
            print(f"{time.asctime()}开始更新最新数据")
            for item in li:
                cursor.execute(sql,item)
            conn.commit()
            print(f"{time.asctime()}更新最新数据完毕")
        else:
            print(f"{time.asctime()}已是最新数据!")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn,cursor)

In [ ]:
def update_country_now():#用时必须判断最新时间为最新数据
    cursor=None
    conn=None
    try:
        li=getdata()[2]
        conn,cursor=get_conn()
        sql ="insert into country_now(update_time,nowConfirm,suspect,nowSevere,confirm,heal,dead,nowConfirm_add,suspect_add,nowSevere_add,confirm_add,heal_add,dead_add) values(%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
        sql_query='select %s=(select update_time from country_now order by id desc limit 1)'
        cursor.execute(sql_query,li[0][0])
        if not cursor.fetchone()[0]:
            print(f"{time.asctime()}开始更新最新数据")
            for item in li:
                cursor.execute(sql,item)
            conn.commit()
            print(f"{time.asctime()}更新最新数据完毕")
        else:
            print(f"{time.asctime()}已是最新数据!")
    except:
        traceback.print_exc()
    finally:
        close_conn(conn,cursor)

In [ ]:
def update_province_history():
    cursor=None
    conn=None
    try:
        li=getdata()[3]
        conn,cursor=get_conn()
        sql ="insert into province_history(ds,province,confirm,confirm_add,heal,dead) values(%s,%s,%s,%s,%s,%s)"
        sql_query='select %s=(select ds from province_history order by id desc limit 1)'
        
        sql2 ="delete from province_history where ds= %s"
        ds2=time.strftime("%Y-%m-%d")
        cursor.execute(sql2,ds2)
        conn.commit()
        
        cursor.execute(sql_query,li[0][0])
        if not cursor.fetchone()[0]:
            print(f"{time.asctime()}开始更新最新数据")
            for item in li:
                cursor.execute(sql,item)
            conn.commit()
            print(f"{time.asctime()}更新最新数据完毕")
        else:
            print(f"{time.asctime()}已是最新数据!")
            
    except:
        traceback.print_exc()
    finally:
        close_conn(conn,cursor)